# Submitting (uploading) data to the JCOIN Data Commons

To upload (and update) a core measure (or other package or file for that matter) to the JDC, one can use the `submit_package_to_jdc` and `map_to_sheepdog` functions (in this case, we use `map_core_measures_to_sheepdog` which runs a set of transforms to go from a validated core measure data package to a sheepdog submission and then uses `map_to_sheepdog` in the `jdc_utils.utils.gen3` module). Sheepdog maps a subset of variables for quicker discovery on the JDC.


In [1]:
from jdc_utils.submission import submit_package_to_jdc,map_core_measures_to_sheepdog
from frictionless import Package
import os

In [2]:
commons_project="TEST"
file_guid="dg.6VTS/22c736c2-8056-4f70-bf78-167402d82ccd"
sheepdog_id="d32e422b-fa6b-46a6-9ffa-893a95b762d0"
zipped_package_path = "../data/core-measures-test.zip"

In [3]:
# from gen3.auth import Gen3Auth
# from gen3.submission import Gen3Submission
# sub = Gen3Submission(Gen3Auth(refresh_file="credentials.json"))
# sub.export_node("JCOIN","TEST","reference_file",fileformat="json")

In [4]:
# create new record - do this only one  time!

In [5]:
submit_package_to_jdc(
    package_path=zipped_package_path,
    commons_project=commons_project,
    file_guid=file_guid,
    sheepdog_file_submitter_id='core-measures-TEST',
    sheepdog_data_type="Interview",
    sheepdog_data_category="Core Measures",
    sheepdog_data_format="ZIP",
    submission_type="create",
)

[2023-04-06 10:29:22,600][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-06 10:29:22,601][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_3403_1680794962' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-06 10:29:22,602][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x000001BF1EF5A5C0>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXIiLCJhdWQiOlsiaHR0cHM6Ly9qY29pbi5kYXRhY29tbW9ucy5pby91c2VyIiwiZ29vZ2xlX3NlcnZpY2VfYWNjb3VudCIsInVzZXIiLCJvcGVuaWQiLCJmZW5jZSIsImRhdGEiLCJnb29nbGVfY3JlZGVudGlhbHMiLCJhZG1pbiIsImdvb2d

NameError: name 'sheepdog_file_metadata' is not defined

In [ ]:
package_path = zipped_package_path.replace(".zip","")+"/data-package.json"
core_measures = Package(package_path)
baseline_df = core_measures.get_resource("baseline").to_pandas()
baseline_df["role_in_project"] = "Client"
timepoints_df = core_measures.get_resource("timepoints").to_pandas()

In [5]:
last_node_output = map_core_measures_to_sheepdog(
    baseline_df=baseline_df,
    timepoints_df=timepoints_df,
    program="JCOIN",
    project="TEST",
    credentials_path="credentials.json",
    # node_list=["demographic_baseline"]
)

Validating records for participant
Validating records for enrollment
Validating records for demographic_baseline
Validating records for time_point
Deleting records for time_point
[2023-04-03 21:28:18,081][WARNING] failed to write token cache file: C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916
[2023-04-03 21:28:18,082][WARNING] [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916.tmp_eraseme_17505_1680575298' -> 'C:\\Users\\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916'
[2023-04-03 21:28:18,083][WARNING] backoff: call gen3.auth._write_to_file(<gen3.auth.Gen3Auth object at 0x000001FE78120820>, C:\Users\kranz-michael/.cache/gen3/token_cache_243db9dca6c478c2a6ee5ce88b9e5916, eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImZlbmNlX2tleV8yMDIwLTAxLTAyVDA2OjM1OjA4WiJ9.eyJwdXIiOiJhY2Nlc3MiLCJpc3MiOiJodHRwczovL2pjb2luLmRhdGFjb21tb25zLmlvL3VzZXI